# LightGBM & Optuna

In [1]:
%load_ext watermark
%watermark -p scikit-learn,optuna

scikit-learn: 0.24.1
optuna      : 2.10.0



## Dataset

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = pd.read_csv('pcaX_without header.csv', header=None).values
y = pd.read_csv('Y.csv', header=None).values.ravel().astype(int)



X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)



#X_test = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/X_test.csv', header=None).values
#y_test = pd.read_csv('https://raw.githubusercontent.com/rasbt/stat451-machine-learning-fs21/main/hw02-starter/dataset/y_test.csv', header=None).values.ravel().astype(int)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)


X_train.shape: (16268, 5)
y_train.shape: (16268,)
X_test.shape: (6973, 5)
y_test.shape: (6973,)


In [3]:
# X_train

In [4]:
# y_train

In [5]:
X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid/Test sizes:', y_train.shape[0], y_valid.shape[0])

Train/Valid/Test sizes: 16268 3254


## My Model

### LightGBM & Optuna

In [6]:
# !pip install optuna
# !pip install lightgbm

In [7]:
import numpy as np
import optuna
from optuna.integration import LightGBMPruningCallback

import lightgbm

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import warnings

warnings.filterwarnings("ignore", category=UserWarning)
#optuna.logging.set_verbosity(optuna.logging.WARNING)


def objective(trial, X_train, y_train, cv=5):
    
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10, 100, 1000, 10000]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.01]),
    }
    
    cv_iterator = StratifiedKFold(n_splits=cv, shuffle=True, random_state=123)

    cv_scores = np.zeros(cv)
    for idx, (train_sub_idx, valid_idx) in enumerate(cv_iterator.split(X_train, y_train)):
        
        X_train_sub, X_valid = X_train[train_sub_idx], X_train[valid_idx]
        y_train_sub, y_valid = y_train[train_sub_idx], y_train[valid_idx]

        model = lightgbm.LGBMClassifier(objective="multi_logloss", **param_grid)
        model.fit(
            X_train_sub,
            y_train_sub,
            eval_set=[(X_valid, y_valid)],
            eval_metric="multi_logloss",
            verbose=-1,
            early_stopping_rounds=50,
            callbacks=[
                LightGBMPruningCallback(trial=trial, metric="multi_logloss")
            ],  # Add a pruning callback to eliminate unpromising candidates
        )
        preds = model.score(X_valid, y_valid)
        
        cv_scores[idx] = preds

    return 1-np.mean(cv_scores)

In [8]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")

def func(trial):
    return objective(trial, X_train, y_train)

study.optimize(func, n_trials=10);

[I 2021-12-05 14:05:27,634] A new study created in memory with name: LGBM Classifier
[I 2021-12-05 14:06:51,814] Trial 0 finished with value: 0.347246520681302 and parameters: {'n_estimators': 10000, 'learning_rate': 0.01}. Best is trial 0 with value: 0.347246520681302.
[I 2021-12-05 14:06:53,122] Trial 1 finished with value: 0.5124170190591408 and parameters: {'n_estimators': 10, 'learning_rate': 0.01}. Best is trial 0 with value: 0.347246520681302.
[I 2021-12-05 14:08:12,713] Trial 2 finished with value: 0.347246520681302 and parameters: {'n_estimators': 10000, 'learning_rate': 0.01}. Best is trial 0 with value: 0.347246520681302.
[I 2021-12-05 14:09:36,958] Trial 3 finished with value: 0.347246520681302 and parameters: {'n_estimators': 1000, 'learning_rate': 0.01}. Best is trial 0 with value: 0.347246520681302.
[I 2021-12-05 14:10:52,917] Trial 4 finished with value: 0.347246520681302 and parameters: {'n_estimators': 10000, 'learning_rate': 0.01}. Best is trial 0 with value: 0.34724

In [9]:
print(f"\tBest value: {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value: 0.34725
	Best params:
		n_estimators: 10000
		learning_rate: 0.01


In [10]:
model = lightgbm.LGBMClassifier(objective="multi_logloss", **study.best_params)
model.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.01, n_estimators=10000,
               objective='multi_logloss')

In [11]:
print(f"Training Accuracy: {model.score(X_train, y_train):0.3f}")
print(f"Test Accuracy: {model.score(X_test, y_test):0.3f}")

Training Accuracy: 1.000
Test Accuracy: 0.659


In [12]:
%timeit model.fit(X_train, y_train) #training(fitting) time
%timeit model.score(X_valid, y_valid) #test time

1min 38s ± 8.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
20.6 s ± 2.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
